In [1]:
import time
import os
import torch
from torchvision import transforms
import random
import numpy as np
import yaml
from PIL import Image
from experiment import VAEXperiment
from models import *


import gymnasium as gym
from gymnasium import ObservationWrapper
from gymnasium.wrappers import PixelObservationWrapper, FrameStack
from gymnasium.spaces import Box, Discrete



from stable_baselines3 import SAC, PPO, A2C
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.callbacks import CallbackList, CheckpointCallback, EvalCallback
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
from gymnasium import ObservationWrapper
from gymnasium.wrappers import PixelObservationWrapper, FrameStack
from gymnasium.spaces import Box, Discrete
import numpy as np
import yaml
from experiment import VAEXperiment
from models import *
import torch
from torchvision import transforms
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv

class AddGaussianNoise(object):
    def __init__(self, mean=0., std=0.1):
        self.std = std
        self.mean = mean
        
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    
    def __repr__(self):
        return self.__class__.__name__ + '(mean={0}, std={1})'.format(self.mean, self.std)

#ideas
#maybe use wrapper to catch and save frames during RL model learning
#alternative is to use the model afterwards to generate frames during a test run
#
#??return mu, std or sample or just mu??
#
class VAE_ENC(ObservationWrapper):
    def __init__(self, env, vae, latent_dim,
                 mean=0,std=0.1,
                 size=(64,64)):
        super().__init__(env)
        #new obs space with std
        #self.observation_space = Box(shape=(2, latent_dim), low=-np.inf, high=np.inf)
        #just mean
        self.observation_space = Box(shape=(latent_dim,), low=-np.inf, high=np.inf)
        
        self.vae = vae
        #transforms
        self.mean = mean
        self.std = std
        self.size = size
        
        
        
    def observation(self, obs):
        #get frame
        #print(obs)
        frame = obs['pixels']#.to('cuda')
        #transform for VAE
        val_transforms = transforms.Compose([transforms.ToTensor(),
        #transforms.RandomHorizontalFlip(),
        AddGaussianNoise(self.mean, self.std),
        transforms.Resize(self.size),
        #transforms.Grayscale(),
        #transforms.Normalize(self.mean, self.std),
        ])
        frame = val_transforms(frame) #(c,h,w)
        frame = torch.unsqueeze(frame, 0)#.to(self.device) #make it (1,c,h,w)
        enc = self.vae.encode(frame)    
        enc = np.array([tensor.detach().cpu().numpy() for tensor in enc])
        #with std
        #enc = np.array([enc[0][0], enc[1][0]]) ## mu, std #  give only mu?
        #just mean
        enc = np.array(enc[0][0])
        return enc
    
def get_vae(version='version_0',log_directory='logs/BCE_sum_VAE/MSSIMVAE/'):

    model_path=log_directory+'/'+version+'/hparams.yaml'
    ckpt_path=log_directory+'/'+version+'/checkpoints/last.ckpt'

    config = yaml.safe_load(open(model_path))
    model = vae_models[config['model_params']['name']](**config['model_params'])
    ckpt = torch.load(ckpt_path)
    experiment = VAEXperiment(model, config['exp_params'])
    experiment.load_state_dict(ckpt['state_dict'])      
    vae = experiment.model
    return vae

In [3]:
data_name='init3_random'
save_path='Data/MountainCar/'+data_name+'/'


env = gym.make("MountainCarContinuous-v0",
                render_mode ='rgb_array')

num_of_episodes=10
i=0
for episode in range(num_of_episodes):
    observation, info = env.reset()
    done = False
    while not done: 
        action= env.env.action_space.sample()
        observation, reward, terminated, truncated, info = env.step(action)
        #state, reward, terminated, truncated, info = env.step(action)
        #print(terminated, truncated)
        if terminated:
            done = True
        if truncated:
            done = True
        current_frame = env.render()

        i+=1
        im = Image.fromarray(np.array(current_frame))
        im.save(save_path+data_name+'_'+str(i)+'.jpeg')

In [36]:
vae = get_vae(version='version_13',log_directory='logs/MountainCar/BCE_sum_VAE_2/MSSIMVAE/')

In [37]:
env = gym.make("MountainCarContinuous-v0",
                render_mode ='rgb_array')
seed = 42
env.reset(seed=seed)
env = PixelObservationWrapper(env)
env = VAE_ENC(env, vae, 2)
env = FrameStack(env, num_stack=2)
env = Monitor(env)


In [38]:
agent_name = "vae_2_v13_agent_3_A2C_best_agent_data"

env_name = "MountainCarContinuous-v0"

models_dir = f"RLmodels/"+env_name+'/'+agent_name
logdir = f"RLlogs/"+env_name+'/vae_A2C'

RL_train_steps = 1000000

callback = CheckpointCallback(save_freq=10000, save_path=models_dir) 

In [39]:
# Tuned
agent = A2C(

    env = env,
    n_steps= 100,
    
    policy='MlpPolicy',
    ent_coef= 0.0,
    use_sde=True,
    sde_sample_freq = 16,
    policy_kwargs= dict(log_std_init=0.0, ortho_init=False),
    tensorboard_log=logdir)

In [40]:
agent.learn(total_timesteps=RL_train_steps, 
            callback=callback,
            tb_log_name=agent_name)

In [4]:
from pathlib import Path

agent_name = "vae_2_v11_agent_1_A2C_long"

env_name = "MountainCarContinuous-v0"

models_dir = f"RLmodels/"+env_name+'/'+agent_name
model_dir = Path(models_dir)
model_list = sorted([f for f in model_dir.iterdir() if f.suffix == '.zip'])
print(len(model_list))

50


In [10]:
str(model_list[0])[55:-4]

'rl_model_100000_steps'

In [11]:
model_list[0]

PosixPath('RLmodels/MountainCarContinuous-v0/vae_2_v6_agent_3_A2C/rl_model_100000_steps.zip')

In [12]:
iteration = '3'

for model in model_list:
    model = str(model)[55:-4] #[43:-4]
    print(model)
    agent= PPO.load(models_dir+'/'+model)
    #save_path='Data/MountainCar/vae_A2C_'+iteration+'_rl_model'
    data_name='vae_A2C_'+iteration+'_'+model+'_'
    num_of_episodes=8
    i=0
    for episode in range(num_of_episodes):
        observation, info = env.reset()
        done = False
        while not done: 
            action, _states = agent.predict(observation, deterministic=True)
            observation, reward, terminated, truncated, info = env.step(action)
            #state, reward, terminated, truncated, info = env.step(action)
            #print(terminated, truncated)
            if terminated:
                done = True
            if truncated:
                done = True
            current_frame = env.render()

            i+=1
            im = Image.fromarray(np.array(current_frame))
            im.save('Data/MountainCar/init_random/'+data_name+'_'+str(i)+'.jpeg')

rl_model_100000_steps
rl_model_10000_steps
rl_model_20000_steps
rl_model_30000_steps
rl_model_40000_steps
rl_model_50000_steps
rl_model_60000_steps
rl_model_70000_steps
rl_model_80000_steps
rl_model_90000_steps


In [33]:
model = 'rl_model_660000_steps'
agent_name = "vae_2_v12_agent_2_A2C_long"

env_name = "MountainCarContinuous-v0"

models_dir = f"RLmodels/"+env_name+'/'+agent_name

iteration = '2'
print(model)
agent= PPO.load(models_dir+'/'+model)
#save_path='Data/MountainCar/vae_A2C_'+iteration+'_rl_model'
data_name='vae_A2C_'+iteration+'_'+model+'_'


num_of_episodes=80

num_of_img=9990
i=0
for episode in range(num_of_episodes):
    observation, info = env.reset()
    done = False
    while not done: 
        action, _states = agent.predict(observation, deterministic=True)
        observation, reward, terminated, truncated, info = env.step(action)
        #state, reward, terminated, truncated, info = env.step(action)
        #print(terminated, truncated)
        if terminated:
            done = True
        if truncated:
            done = True
        current_frame = env.render()

        i+=1
        im = Image.fromarray(np.array(current_frame))
        im.save('Data/MountainCar/init3_agent_2_v12/'+data_name+'_'+str(i)+'.jpeg')
        if i == num_of_img:
            break

rl_model_660000_steps


In [13]:
from pathlib import Path
imgs_dir ="Data/MountainCar/init3_random"
img_dir = Path(imgs_dir)
img_list = sorted([f for f in img_dir.iterdir() if f.suffix == '.jpeg'])
print(len(img_list))

9990


In [ ]:

i=0
for imp in img_list:
    im = Image.open(r"C:\Users\System-Pc\Desktop\ybear.jpg") 
    im.save('Data/MountainCar/init3_random_and_agent_0/'+data_name+'_'+str(i)+'.jpeg')
    i+=1